In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

import tensorflow
from tensorflow import keras
from keras import layers

from tensorflow.keras import Sequential
import os
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
data_dir ='/content/drive/MyDrive/Colab Notebooks/MNIST_dataset'

In [5]:
training_dir= os.path.join(data_dir, 'mnist_train.csv')
test_dir=os.path.join(data_dir,'mnist_test.csv')

In [6]:
df = pd.read_csv(training_dir)

In [7]:
X_train= df.drop('label',axis=1)
Y_train = df['label']

In [8]:
df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train=X_train.to_numpy()
Y_train=Y_train.to_numpy()
X_train=X_train.reshape(60000,28,28,1)
print(Y_train.shape)

(60000,)


In [10]:
from keras.utils import to_categorical

In [11]:
X_train=X_train/255

Y_train=to_categorical(Y_train)
Y_train.shape


(60000, 10)

In [12]:
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Dropout, Flatten


In [13]:
X_train=np.dstack([X_train] * 3)
X_train = X_train.reshape(-1, 28,28,3)
X_test=np.dstack([X_train] * 3)
X_test = X_train.reshape(-1, 28,28,3)


In [14]:
X_train_padded = np.pad(X_train, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant', constant_values=0)
X_test_padded = np.pad(X_test, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant', constant_values=0)
X_train_padded.shape


(60000, 32, 32, 3)

In [15]:
resnet_model = Sequential()

pretrained_model= ResNet50(include_top=False,
                   input_shape=(32,32,3),
                   pooling='avg',
                   classes=10,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(10, activation='softmax'))

94765736/94765736 [==============================] - 1s 0us/step


In [16]:
from keras.optimizers import Adam
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
resnet_model.fit(X_train_padded,Y_train,epochs=5,batch_size=128)

Epoch 1/5
469/469 [==============================] - 19s 18ms/step - loss: 1.0961 - accuracy: 0.6553
Epoch 2/5
469/469 [==============================] - 9s 19ms/step - loss: 0.6102 - accuracy: 0.8062
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.5116 - accuracy: 0.8369
Epoch 4/5
469/469 [==============================] - 9s 20ms/step - loss: 0.4528 - accuracy: 0.8554
Epoch 5/5
469/469 [==============================] - 8s 17ms/step - loss: 0.4241 - accuracy: 0.8639


In [18]:
# Input shape should be at least (224, 224, 3)
input_shape = (32, 32, 3)

n_resnet_model = Sequential()

# Use the ResNet50 model as a feature extractor
pretrained_model = ResNet50(include_top=False,
                            input_shape=input_shape,
                            pooling='avg',
                            weights='imagenet')

# Freeze the layers of the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

n_resnet_model.add(pretrained_model)

# Flatten layer isn't needed because ResNet50 with pooling='avg' outputs a 1D tensor
# resnet_model.add(Flatten())

n_resnet_model.add(Dense(512, activation='relu'))
n_resnet_model.add(Dense(10, activation='softmax'))

n_resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24641930 (94.00 MB)
Trainable params: 1054218 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [20]:
n_resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
n_resnet_model.fit(X_train_padded,Y_train,epochs=5,batch_size=128)

Epoch 1/5
469/469 [==============================] - 12s 20ms/step - loss: 1.0568 - accuracy: 0.6634
Epoch 2/5
469/469 [==============================] - 8s 18ms/step - loss: 0.6047 - accuracy: 0.8059
Epoch 3/5
469/469 [==============================] - 9s 20ms/step - loss: 0.5007 - accuracy: 0.8389
Epoch 4/5
469/469 [==============================] - 9s 20ms/step - loss: 0.4545 - accuracy: 0.8533
Epoch 5/5
469/469 [==============================] - 12s 26ms/step - loss: 0.4250 - accuracy: 0.8630
